In [1]:
#import mysql
import json
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

## Downloand database

In [2]:
# def connect_to_database():
#     try:
#         with open('secret.txt', 'r') as file:
#             connection = mysql.connector.connect(
#                 host='localhost',
#                 database='magic_cards',
#                 user='root',
#                 password=file.readline()
#             )
#         if connection.is_connected():
#             print("Connected to MySQL database")
#             return connection
#     except mysql.connector.Error as e:
#         print(f"Error while connecting to MySQL: {e}")
#     return None

# def export_to_csv(connection, filename='magic_cards.csv'):
#     query = """
#     SELECT name, colorIdentity, keywords, supertypes
#     FROM cards
#     """
#     df = pd.read_sql(query, connection)
#     df.to_csv(filename, index=False)
#     print(f"Data exported to {filename}")



In [3]:
# Main execution
# connection = connect_to_database()
# if connection:
#     export_to_csv(connection)
#     connection.close()
# else:
#     print("Failed to connect to the database.")

## Analysis portion

In [ ]:
def load_card_data(filename='cards.csv'):
	return pd.read_csv(filename)

def prepare_features(df):
	# Process colors
	color_map = {'W': 'White', 'U': 'Blue', 'B': 'Black', 'R': 'Red', 'G': 'Green'}
	df['colorIdentity'] = df['colorIdentity'].fillna('').apply(lambda x: [color_map.get(c.strip(), c.strip()) for c in x.split(',')] if isinstance(x, str) else [])
	mlb_colors = MultiLabelBinarizer()
	color_features = pd.DataFrame(
		mlb_colors.fit_transform(df['colorIdentity']),
		columns=mlb_colors.classes_,
		index=df.index
	)
	
	# Process keywords
	df['keywords'] = df['keywords'].fillna('').apply(lambda x: [kw.strip() for kw in x.split(',')] if isinstance(x, str) else [])
	mlb_keywords = MultiLabelBinarizer()
	keyword_features = pd.DataFrame(
		mlb_keywords.fit_transform(df['keywords']),
		columns=mlb_keywords.classes_,
		index=df.index
	)
	
	# Process supertypes
	df['is_legendary'] = df['supertypes'].fillna('').str.contains('Legendary')
	
	# Combine features
	features = pd.concat([color_features, keyword_features, df[['is_legendary']]], axis=1)
	
	return features, mlb_colors, mlb_keywords

def cluster_cards(features, n_clusters=10):
	kmeans = KMeans(n_clusters=n_clusters, random_state=69, n_init=10)
	return kmeans.fit_predict(features)

def find_related_cards(df, features, legendary_card_name, top_n=1000):
	legendary_card = df[df['name'] == legendary_card_name].index[0]
	legendary_features = features.iloc[legendary_card].values.reshape(1, -1)
	
	similarities = cosine_similarity(legendary_features, features)
	similar_indices = similarities[0].argsort()[::-1][1:top_n+1]
	
	return df.iloc[similar_indices]

In [5]:
df = load_card_data()
df.drop_duplicates(subset='name', keep="last")
features, mlb_colors, mlb_keywords = prepare_features(df)

# Perform clustering
clusters = cluster_cards(features)
df['cluster'] = clusters

/var/folders/6m/877x85y12tb8t246sy17j4180000gn/T/ipykernel_9263/3459460400.py:2: DtypeWarning: Columns (3,7,12,16,20,23,25,26,31,32,33,35,39,40,41,47,52,53,61,62,66,68) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(filename)


In [6]:
print("Sample of clustered cards:")
print(df[['name', 'colorIdentity', 'keywords', 'is_legendary', 'cluster']].head(5))

Sample of clustered cards:
                name colorIdentity        keywords  is_legendary  cluster
0  Ancestor's Chosen       [White]  [First strike]         False        7
1  Ancestor's Chosen       [White]  [First strike]         False        7
2     Angel of Mercy       [White]        [Flying]         False        7
3     Angel of Mercy       [White]        [Flying]         False        7
4   Angelic Blessing       [White]              []         False        1


In [ ]:
# Find related cards for a legendary creature
legendary_card_name = "Reya Dawnbringer"  # An example from your data
if legendary_card_name in df['name'].values:
	related_cards = find_related_cards(df, features, legendary_card_name)
	pd.set_option('display.max_rows', None)
	print(f"\nCards related to {legendary_card_name}:")
	print(related_cards[['name', 'colorIdentity', 'keywords', 'is_legendary', 'text']])
else:
	print(f"Legendary creature '{legendary_card_name}' not found in the database.")


Cards related to Reya Dawnbringer:
                                                    name   colorIdentity  \
13059                         Teshar, Ancestor's Apostle         [White]   
26166                                Zeriam, Golden Wind         [White]   
20525                         Teshar, Ancestor's Apostle         [White]   
76342                        Mavinda, Students' Advocate         [White]   
26242                                Zeriam, Golden Wind         [White]   
85668                        Mavinda, Students' Advocate         [White]   
79694                         Linvala, Keeper of Silence         [White]   
87725                                        Major Teroh         [White]   
76343                        Mavinda, Students' Advocate         [White]   
49                                      Reya Dawnbringer         [White]   
48                                      Reya Dawnbringer         [White]   
26677                                  Lieutenant Ki

In [8]:
# Print some statistics
print("\nCluster Statistics:")
print(df['cluster'].value_counts())

print("\nMost common keywords:")
print(df['keywords'].explode().value_counts().head(10))

print("\nColor distribution:")
print(df['colorIdentity'].explode().value_counts())


Cluster Statistics:
cluster
0    17077
6    15611
2    13611
4    10813
3    10324
1    10252
7     6593
8     6157
9     4250
5     3004
Name: count, dtype: int64

Most common keywords:
keywords
             57285
Flying        8811
Enchant       3294
Trample       2441
Haste         1784
Vigilance     1675
Equip         1463
Mill          1427
Flash         1384
Scry          1345
Name: count, dtype: int64

Color distribution:
colorIdentity
Green    22038
Black    21896
White    21690
Blue     21666
Red      21650
         10813
Name: count, dtype: int64


In [ ]:
cards = pd.read_csv('cards.csv')
cards = cards[cards.availability.str.contains('paper')].drop_duplicates(subset='name', keep='last')
cards = cards[['name', 'colorIdentity', 'keywords', 'text', 'edhrecRank', 'manaValue', 'uuid', 'availability', 'isOnlineOnly', 'isTextless']]
print(len(cards.index))
with open('Keywords.json') as json_file:
	keywords = json.load(json_file)

#load dict of all keywords
keywords = pd.DataFrame({'keywords': keywords})

# Perform K-Means clustering based on the EDHRec Rank attribute

# Get range of values in legal card set
# valid_cards = cards[]
print(set(cards[cards.name == 'Shalai and Hallar']['colorIdentity'].split(', ')))

# Use elbow-knee to select best K

# Perform Apriori association rule mining based on the rules text similarity score

#split rules text into all possible permutations, then do this for each candidate and compare amount of matching options to increase support
# def split_rules(rules_text):
#     splits = []
#     #split at first occurrence and store, then second, and so on. then iterate and do it again
#     parsed_rules = rules_text.split("\\n")[-1].split(" ")

#     for i in range(len(parsed_rules)):
#         cut_rules = " ".join(parsed_rules[:len(parsed_rules) - i])
#         for j in range(len(cut_rules.split(" "))):
#             #split at last occurrence, then take the first value in split to get full string
#             sliced_rules = cut_rules.split("\\n")[-1].split(" ", j)
#             # for j in range(len(bruh) - i):
#             splits.append(sliced_rules[-1])
#     return splits

# def apriori(ref_name, related_cards):
#     # ref_name is L_1
#     candidates = []
#     rules_matrix = []

#     for text in related_cards['text'].tolist():
#         rules_matrix.append(split_rules(text))

	# Get initial comparison score
	  
	# While k < 63, append candidates
	
	# Run for each cluster starting with most related

	# Generate candidate based on highest average score against all in list
	# Iterate over existing list size k
	# Take average and find max among all cards


# cell_value = cards.where(cards=='Reya Dawnbringer').dropna(how='all').dropna(axis=1)
# # print(related_cards[['name', 'text']])
# #increase support value based on shared keywords, shared terms in rules text, and triggers
# pd.set_option('display.max_colwidth', None)
# #print(cards.loc[cell_value.index[-1], 'text']) 
# # rules = split_rules(cards.loc[cell_value.index[-1], 'text'])

# candidates = apriori(legendary_card_name, related_cards)
# for i in candidates: print(i)

/var/folders/6m/877x85y12tb8t246sy17j4180000gn/T/ipykernel_9263/733514656.py:1: DtypeWarning: Columns (3,7,12,16,20,23,25,26,31,32,33,35,39,40,41,47,52,53,61,62,66,68) have mixed types. Specify dtype option on import or set low_memory=False.
  cards = pd.read_csv('cards.csv')


29639


AttributeError: 'set' object has no attribute 'split'

In [ ]:
import pandas as pd
import json
import numpy as np
import sys
import math

# Perform K-Means clustering on the set of valid cards using EDHRec Rank to determine distance
def k_means(k, cards):
	centroids = []

	# Maybe update to force into range of quarters of true range?
	for i in range(k):
		centroids.append(np.random.randint(valid_cards['edhrecRank'].min(), valid_cards['edhrecRank'].max()))

	print(centroids)

	# Compute minimum distance for each card and assign accordingly
	assignments = []
	for i in range(k):
		cluster = []
		assignments.append(cluster)

	# print(centroids.index(max(centroids)))
	
	for index in cards.index:
		min_distance = sys.maxsize
		assigned_centroid_index = centroids.index(max(centroids))

		for rank in centroids:
			if abs(cards.loc[index]['edhrecRank'] - rank) < min_distance:
				assigned_centroid_index = centroids.index(rank)
				min_distance = abs(cards.loc[index]['edhrecRank'] - rank)
		assignments[assigned_centroid_index].append(index)

	print(k, len(centroids), len(assignments))

	# Compute new centroids based on averages of cards in cluster
	clustered_cards = []
	for i in range(k):
		print(i)
		print(centroids[i])
		print(assignments[i])
		
		sum = 0
		for index in assignments[i]:
			sum += cards.loc[index]['edhrecRank']
		
		centroids[i] = sum / len(assignments[i])
		print('new centroid: {}'.format(centroids[i]))
		clustered_cards.append((centroids[i], assignments[i]))

	return sorted(clustered_cards)

cards = pd.read_csv('cards.csv')
cards = cards[cards.availability.str.contains('paper')].drop_duplicates(subset='name', keep='last')
cards = cards[['name', 'colorIdentity', 'keywords', 'text', 'edhrecRank', 'manaValue', 'uuid', 'availability', 'isOnlineOnly', 'isTextless']]
cards['edhrecRank'] = cards['edhrecRank'].fillna(cards['edhrecRank'].max())
print(len(cards.index))
with open('Keywords.json') as json_file:
	keywords = json.load(json_file)

#load dict of all keywords
keywords = pd.DataFrame({'keywords': keywords})

# Perform K-Means clustering based on the EDHRec Rank attribute

# Get range of values in legal card set
# valid_cards = cards[]
print(set(cards[cards.name == 'Shalai and Hallar']['colorIdentity'].iloc[0].split(', ')))

commander_identity = set(cards[cards.name == 'Shalai and Hallar']['colorIdentity'].iloc[0].split(', '))

# Get all indices of compliant cards
sum = 0
indices = []
# print(cards.index)

# print(cards.index[0])
# print(cards.loc[2684]['colorIdentity'])
for row in cards.index:
	if type(cards.loc[row]['colorIdentity']) is float:
		identity = set()
	else:
		identity = set(cards.loc[row]['colorIdentity'].split(', '))
	if identity <= commander_identity:
		indices.append(row)
		sum += 1
	
# print(sum)

valid_cards = cards[cards.index.isin(indices)]

# print(valid_cards['edhrecRank'].min(), valid_cards['edhrecRank'].max(), valid_cards['edhrecRank'].mean())

k_means(4, valid_cards)

C:\Users\Jackie\AppData\Local\Temp\ipykernel_20420\2558551675.py:53: DtypeWarning: Columns (3,7,12,16,20,23,25,26,31,32,33,35,39,40,41,47,52,53,61,62,66,68) have mixed types. Specify dtype option on import or set low_memory=False.
  cards = pd.read_csv('cards.csv')


29639
{'W', 'G', 'R'}
[18948, 23493, 8018, 22734]
4 4 4
0
18948
[2826, 2854, 2870, 2994, 4054, 4058, 4062, 4063, 4112, 4113, 4126, 4127, 4143, 4144, 4153, 4168, 4169, 4174, 4187, 4193, 4202, 4208, 4211, 7577, 7586, 7619, 7620, 7625, 7667, 7679, 7680, 7681, 7692, 7694, 7700, 7713, 7726, 7736, 7788, 7791, 7806, 7807, 7809, 7824, 7828, 7832, 7834, 7847, 7857, 7862, 7863, 7864, 7870, 7881, 7885, 7887, 7898, 8240, 8252, 8253, 8254, 8255, 8264, 8267, 8272, 8280, 8281, 8286, 8380, 8386, 8399, 8403, 8414, 8419, 8425, 8428, 8429, 8435, 8441, 8443, 8445, 8458, 8461, 8493, 8510, 8560, 8580, 8585, 8587, 8588, 8589, 8593, 8609, 8690, 8691, 8703, 8718, 8807, 8811, 8821, 8828, 8835, 8837, 8842, 8844, 8846, 8850, 8875, 8908, 8926, 8928, 8934, 8966, 8968, 9361, 9375, 9376, 9378, 9442, 9446, 9456, 9458, 9473, 9481, 9483, 9484, 9486, 9490, 9492, 9534, 9563, 9603, 9612, 9697, 9711, 9714, 9739, 9741, 9747, 9769, 9771, 9773, 9775, 9778, 9780, 9781, 9783, 9789, 9965, 9968, 10013, 10044, 10153, 10158, 10162, 

[(np.float64(6664.990876636255),
  [2684,
   2686,
   2688,
   2690,
   2692,
   2694,
   2696,
   2698,
   2700,
   2702,
   2704,
   2706,
   2806,
   2808,
   2810,
   2812,
   2814,
   2816,
   2818,
   2820,
   2822,
   2824,
   2828,
   2830,
   2832,
   2834,
   2836,
   2838,
   2840,
   2842,
   2844,
   2846,
   2848,
   2850,
   2852,
   2856,
   2858,
   2860,
   2862,
   2864,
   2866,
   2868,
   2872,
   2876,
   2896,
   2932,
   2946,
   2966,
   2968,
   2970,
   2972,
   2974,
   2976,
   2978,
   2980,
   2982,
   2984,
   2986,
   2988,
   2990,
   2992,
   2996,
   2998,
   3000,
   4123,
   4131,
   4138,
   4140,
   4155,
   4161,
   4171,
   4173,
   4197,
   4206,
   4207,
   7405,
   7437,
   7440,
   7575,
   7576,
   7579,
   7580,
   7584,
   7585,
   7587,
   7617,
   7621,
   7622,
   7624,
   7626,
   7629,
   7632,
   7634,
   7639,
   7640,
   7652,
   7655,
   7656,
   7657,
   7658,
   7659,
   7661,
   7662,
   7663,
   7666,
   7668,
   7669,
   7